In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
class ARIMA:
    def __init__(self, p, d, q):
        self.p = p # AR order
        self.d = d # differencing order
        self.q = q # MA order
        self.ar_params = None # AR coefficients
        self.ma_params = None # MA coefficients
        self.mean = 0 # mean of differenced data
    
    def difference(self, data):
        diff_data = data.copy()
        for _ in range(self.d):
            diff_data = np.diff(diff_data)
        return diff_data

    def inverse_difference(self, last_val, forecast):
         for _ in range(self.d):
            forecast = np.cumsum(np.insert(forecast, 0, last_val))
            last_val = forecast[0]
         return forecast

    def train(self, data):
        diff_data = self.difference(data)
        self.mean = np.mean(diff_data)
        diff_data_centered = diff_data - self.mean
        if self.p > 0:
            X_ar = [diff_data_centered[i-self.p:i][::-1] for i in range(self.p, len(diff_data_centered))]
            y_ar = diff_data_centered[self.p:]
            self.ar_params = np.linalg.solve(np.dot(np.transpose(X_ar), X_ar), np.dot(np.transpose(X_ar), y_ar))
        else:
            self.ar_params = np.array([])
        if self.q > 0:
            residuals = diff_data_centered[self.p:] - np.dot(X_ar, self.ar_params) if self.p > 0 else diff_data_centered
            X_ma = [residuals[i-self.q:i][::-1] if i >= self.q else np.zeros(self.q) for i in range(len(residuals))]
            y_ma = residuals
            self.ma_params = np.linalg.solve(np.dot(np.transpose(X_ma), X_ma), np.dot(np.transpose(X_ma), y_ma))
        else:
            self.ma_params = np.array([])


    def predict(self, data, steps):
        diff_data = self.difference(data)
        diff_data_centered = diff_data - self.mean
        history_ar = list(diff_data_centered[-self.p:] if self.p > 0 else [])
        history_ma = [0] * self.q if self.q > 0 else []
        forecasts = []

        for _ in range(steps - 1):
            ar_pred = np.dot(history_ar, self.ar_params) if self.p > 0 else 0
            ma_pred = np.dot(history_ma, self.ma_params) if self.q > 0 else 0
            prediction = ar_pred + ma_pred + self.mean
            forecasts.append(prediction)
            history_ar.append(prediction)
            if len(history_ar) > self.p:
                history_ar.pop(0)
            history_ma.append(diff_data_centered[-1] - ar_pred - ma_pred)
            if len(history_ma) > self.q:
                history_ma.pop(0)
            diff_data_centered = np.append(diff_data_centered, prediction)

        return self.inverse_difference(data[-1], np.array(forecasts))


# Load the datasets
plt.figure(figsize=(10, 5))
p_df2 = pd.read_csv("prices_round_1_day_-2.csv", sep=";")
p_df1 = pd.read_csv("prices_round_1_day_-1.csv", sep=";")
p_df0 = pd.read_csv("prices_round_1_day_0.csv", sep=";")

p_df1.loc[:, "timestamp"] = p_df1["timestamp"] + 1000000
p_df0.loc[:, "timestamp"] = p_df0["timestamp"] + 2000000
p_df = pd.concat([p_df2, p_df1, p_df0])

kelp_df = p_df[p_df["product"] == "KELP"].copy()
si_df = p_df[p_df["product"] == "SQUID_INK"].copy()
train_arr = kelp_df["mid_price"].to_numpy()
kelp_mod = ARIMA(p=5, d=1, q=5)
kelp_mod.train(train_arr)

'''
#replace Nan values with 0 in bid_volume and ask_volume
si_df["bid_volume_1"].fillna(0, inplace=True)
si_df["bid_volume_2"].fillna(0, inplace=True)
si_df["bid_volume_3"].fillna(0, inplace=True)
si_df["ask_volume_1"].fillna(0, inplace=True)
si_df["ask_volume_2"].fillna(0, inplace=True)
si_df["ask_volume_3"].fillna(0, inplace=True)
si_df['bid_vol'] = si_df['bid_volume_1'] + si_df['bid_volume_2'] + si_df['bid_volume_3']
si_df['ask_vol'] = si_df['ask_volume_1'] + si_df['ask_volume_2'] + si_df['ask_volume_3']

volume_scale = 0.3 
plt.xlim(0, 100000)
plt.scatter(si_df["timestamp"], si_df["mid_price"], s=0.1, c="blue", label="SQUID_INK")
plt.xlim(0, 100000)

# For each point, add a line for bid_volume (upward)
for i, row in si_df.iterrows():
    if row['timestamp'] > 100000:
        break
    plt.vlines(x=row['timestamp'], 
               ymin=row['mid_price'], 
               ymax=row['mid_price'] + row['bid_vol'] * volume_scale, 
               color='green', 
               linewidth=.5,
               alpha=0.7)

# For each point, add a line for ask_volume (downward)
for i, row in si_df.iterrows():
    if row['timestamp'] > 100000:
        break
    plt.vlines(x=row['timestamp'], 
               ymin=row['mid_price'] - row['ask_vol'] * volume_scale, 
               ymax=row['mid_price'], 
               color='red', 
               linewidth=.5,
               alpha=0.7)

si_df.head()
'''
print("ar_params")
print(kelp_mod.ar_params)
print("ma_params")
print(kelp_mod.ma_params)
print("mean")
print(kelp_mod.mean)

ar_params
[-0.71823891 -0.50449378 -0.35583069 -0.22313214 -0.11187229]
ma_params
[-0.01331738 -0.0184114  -0.02800807 -0.03931557 -0.05538944]
mean
0.001116703890129671


<Figure size 1000x500 with 0 Axes>

In [ ]:
"""
train_df = kelp_df[kelp_df["timestamp"] < 2000000]
test_df = kelp_df[kelp_df["timestamp"] >= 2000000]
train_df = train_df["mid_price"].to_numpy()
test_df = test_df["mid_price"].to_numpy()

model = ARIMA(p=5, d=1, q=5) 
model.train(train_df)

slices = []
for i in range(3000):
    start = np.random.randint(0, len(test_df) - 8)
    slices.append(test_df[start:start + 8])
e = []
for s in slices:
    pred = model.predict(s[:6], 2).astype(np.int64)
    print("Pred: ", pred)
    print("Truth: ", s[6:].astype(np.int64))
    e.append(np.sum(s[6:].astype(np.int64) - pred))
avg_e = np.mean(np.abs(e))
print("Average Error: ", avg_e)
"""